## Abre y examina el archivo de datos.

In [12]:
#Importar bibliotecas 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split  
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score  
from sklearn.dummy import DummyClassifier

In [13]:
# Carga el archivo de datos en un DataFrame
try:
    df = pd.read_csv('users_behavior.csv')
except: 
    df = pd.read_csv('/datasets/users_behavior.csv')

In [14]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


No hay datos ausentes, el objetivo de los datos es la columna is_ultra y las columnas carateristicas son todas menos is_ultra

## Segmentar los datos

In [15]:
#Seleccionamos las columnas de objetivo y las caracteristicas 
features = df.drop('is_ultra', axis = 1)
target = df['is_ultra']

In [16]:
#Segmentamos los datos en 3:1:1
#Segmentamos los datos para entrenamiento 
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.40, random_state=12345)

In [17]:
#Segmentamos los datos para validación y prueba
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.50, random_state=12345)

In [18]:
#Comprobamos la distribucón 
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


Segmentamos los datos en 3:1:1, eso quiere decir 60% para entrenamiento, 20% para validación y 20% para el modelo de prueba

## Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

Utilisaremos un modelo de clasificación, ya que el modelo solo tiene que predecir entre 0 y 1 

+ Probaremos el modelo con los datos de prueba 

### Decision Tree Classifier

In [19]:
from sklearn.tree import DecisionTreeClassifier   
# < crea un bucle para max_depth de 1 a 5 > 
for depth_dtc in range(1, 6): 
# < crea un modelo, especificar max_depth=depth >     
        model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth_dtc) 
# < entrena el modelo >     
        model_dtc.fit(features_train, target_train) 
# < encuentra las predicciones usando el conjunto de validación > 
        predictions_dtc = model_dtc.predict(features_valid) 
     
        print("max_depth =", depth_dtc, ": ", end='') 
        print(accuracy_score(target_valid, predictions_dtc))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Se entrena el modelo con Decision Tree Classifier y tenemos una exactitud de 78.5% con una profundidad de arbol de 3

### Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier  
best_score_rfr = 0 
best_est_rfr = 0 
# selecciona el rango del hiperparámetro 
for est_rfr in range(1, 10):  
# configura el número de árboles 
    model_rfr = RandomForestClassifier(random_state=54321, n_estimators=est_rfr)  
# entrena el modelo en el conjunto de entrenamiento 
    model_rfr.fit(features_train, target_train) 
# calcula la puntuación de exactitud en el conjunto de validación     
    score_rfr = model_rfr.score(features_valid, target_valid)  
    if score_rfr > best_score_rfr: 
        best_score_rfr = score_rfr # guarda la mejor puntuación de exactitud en el conjunto de validación 
        best_est_rfr = est_rfr #guarda el número de estimadores que corresponden a la mejor puntuación de exactitud 
        
print("Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est_rfr, best_score_rfr))

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 8): 0.7744945567651633


Se entrena el modelo con Random Forest Classifier y tenemos una exactitud de 77.9% con un numero de estimaciones de 50

Podemos observar que de los dos modelos de clasificación, con el que mayor exactitud obtuvimos fue el de Decision Tree Classifier, con el que trabajaremos

## Comprueba la calidad del modelo usando el conjunto de prueba

In [21]:
# Creamos el modelo final 
model_final = DecisionTreeClassifier(random_state=12345, max_depth=3)  
#Entrenamos el modelo 
model_final.fit(features_train, target_train) 

test_predictions = model_final.predict(features_test) 
def accuracy(answers, predictions):  
    correct = 0  
    for i in range(len(answers)):  
        if predictions[i] == answers[i]:  
            correct = correct +1  
    return correct / len(answers)  
print('Exactitud:', accuracy(target_test.values, test_predictions))

Exactitud: 0.7791601866251944


Nuestro modelo tiene una exactitud del 78% de exactitud, con una profundidad de 3, esto quier decir que 8 de cada 10 estan correctas, de igual manera cabe de observar que con el modelo de prueba la exactitud bajo de 78.5% a 77.9%

## Prueba de cordura al modelo. 

In [11]:
# Creamos el modelo de cordura con DummyClassifier
dummy_df = DummyClassifier(strategy="most_frequent")  
#Entrenamos el modelo 
dummy_df.fit(features_train, target_train) 

dummy_predictions = dummy_df.predict(features_test) 

print('Exactitud:', accuracy(target_test.values, dummy_predictions))

Exactitud: 0.6842923794712286


Podemos observar la exactitud del modelo Dummy (68.4%) es menor a la exactitud de nuestro mejor modelo que es 77.9%, nuestro modelo pasa la prueba de cordura